This is a practice assignment 2 for Course 8/12 (Data visualization) in Data Scientist Certification

In [1]:
# Importing the neccessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import callback

In [2]:
import datetime as dt

In [3]:
# Crating the app and reading data into pandas.DataFrame
app = dash.Dash(__name__)
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

In [4]:
# making sure data is read correctly
df.head()

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced
0,NSW,1/4/2005,8.68000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R
1,NSW,1/5/2005,16.61125,322.475000,62.362500,85.500000,8.088793,65.428571,8,R
2,NSW,1/6/2005,5.52000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R
3,NSW,1/7/2005,6.26400,313.870000,33.800000,92.200000,7.529940,56.700000,5,R
4,NSW,1/8/2005,5.40000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R


In [5]:
# Transforming 'Date' column to a more suitable datatype and creating 2 new columns
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month_name()
df['Year'] = df['Date'].dt.year

In [6]:
df['Region'].unique()

array(['NSW', 'NT', 'QL', 'SA', 'TA', 'VI', 'WA'], dtype=object)

In [7]:
df.dtypes

Region                                         object
Date                                   datetime64[ns]
Estimated_fire_area                           float64
Mean_estimated_fire_brightness                float64
Mean_estimated_fire_radiative_power           float64
Mean_confidence                               float64
Std_confidence                                float64
Var_confidence                                float64
Count                                           int64
Replaced                                       object
Month                                          object
Year                                            int32
dtype: object

In [8]:
# time to create the skeleton of our app
app.layout = html.Div(children=[
    html.H1('Australia Wildfire Dashboard', style={'textAlign':'center',
                                                  'font-size':'40'}),
    html.Div(['Input Region:, ',
             dcc.Dropdown(['New South Wales', 'Northern Territory', 'Queensland', 'South Australia', 'Tasmania', 'Victoria', 'Western Australia'], 'Queensland', 
                         id='input-region')]),
    html.Div(['Input Year:', 
             dcc.Input(id='input-year', value='2005', type='number')]),
    html.Br(),
    # Piechart
    html.Div([
        html.Div([dcc.Graph(id='output-pie')], style={'display': 'flex'}),
        html.Div([dcc.Graph(id='output-bar')], style={'display': 'flex'})], style={'display': 'flex'})
])
# Creating callback for interactivity

@app.callback(Output(component_id='output-pie', component_property='figure'),
              Output(component_id='output-bar', component_property='figure'),
             Input(component_id='input-region', component_property='value'),
             Input(component_id='input-year', component_property='value'))
def create_dashboard(sel_region, sel_year):
    sel_year = int(sel_year)
    d = {'New South Wales':'NSW', 'Northern Territory':'NT', 'Queensland':'QL', 'South Australia':'SA', 'Tasmania':'TA', 'Victoria':'VI', 'Western Australia':'WA'}
    sel_region = d[sel_region]
    new_df = df[(df['Year'] == sel_year) & (df['Region'] == sel_region)].groupby('Month')['Estimated_fire_area'].mean().reset_index()
    pie_chart = px.pie(values=new_df['Estimated_fire_area'], names=new_df['Month'], title='... : Monthly average estimated fire area in year ...')
    # Creating barplot
    new_df = df[(df['Year'] == sel_year) & (df['Region'] == sel_region)].groupby('Month')['Count'].mean().reset_index()
    bar_plot = px.bar(x=new_df['Month'], y=new_df['Count'], title='...: Average Count of Pixels for Presumed Vegetation Fires in the year ...')
    return [pie_chart, bar_plot]
# Run the app
if __name__ == '__main__':
    app.run(debug=True)

# To improve - maybe instead of input window for the year(as requested) make a list/ create error messages if invalid years are chosen.